In [1]:
import sqlite3
import requests
import re
from bs4 import BeautifulSoup, Comment
import json
import pandas as pd
import time
import folium
import matplotlib.pyplot as plt
import urllib.request
from selenium import webdriver
import random

import traceback

In [308]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, f1_score, recall_score


In [2]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer #WBOT
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer #these three are for visualization
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn import metrics
from matplotlib import cm
import numpy as np




import string, re
from nltk.stem import WordNetLemmatizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GMoneyMan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GMoneyMan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GMoneyMan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

### Cleaning the Dataframe

In [4]:
beer_df = pd.read_csv(r'C:\Users\GMoneyMan\Documents\Flatiron\capstone\csv_data\beer_w_reviews.csv')

In [5]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,drop_me,url,review
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,NaN,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev..."
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,NaN,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,NaN,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%..."
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.20,NaN,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,NaN,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...
...,...,...,...,...,...,...,...,...,...
7339,Fruit and Field,Radler,Occy's Brewery,4.80,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/2128...,NaN
7340,Fruit and Field,Alan-A-Dale Apple Ale,Robin Hood Brewing Co. / Home D Pizzeria,5.00,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/4182...,NaN
7341,Fruit and Field,Tussle In the Hay,Back Unturned Brewing Company,7.90,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/5844...,NaN
7342,Fruit and Field,Strawberry Chocolate Cheesecake Sour,Barn Town Brewing,5.90,0,0.00,NaN,https://www.beeradvocate.com/beer/profile/4760...,NaN


In [6]:
beer_df.isna().sum()

beer_style         0
beer_name          0
brewery_name       0
abv              144
no_ratings       144
avg_rating       144
drop_me         7344
url              144
review          1799
dtype: int64

In [7]:
#dropping the column that has no data
beer_df.drop('drop_me', axis = 1, inplace = True)

In [8]:
#dropping the rows that have no reviews
beer_df.dropna(subset=['review'], inplace = True)

In [9]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,"5,915",3.42,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev..."
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,"4,029",3.19,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,"3,992",4.41,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%..."
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,"3,841",3.20,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,"3,799",3.91,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...
...,...,...,...,...,...,...,...,...
5651,Fruit and Field,Carney Juice,Independent Republic Brewing Co.,4.00,1,4.00,https://www.beeradvocate.com/beer/profile/5671...,['PaleAleWalt from New York4/5\xa0\xa0rDev 0%l...
5652,Fruit and Field,Cherry Ace,The Blue Elephant Craft Brew House,4.80,1,3.31,https://www.beeradvocate.com/beer/profile/3446...,['KrisDLSmith from Canada (ON)3.31/5\xa0\xa0rD...
5653,Fruit and Field,Cherry Stout,Cottontown Brew Lab,?,1,3.41,https://www.beeradvocate.com/beer/profile/5210...,['RochefortChris from North Carolina3.41/5\xa0...
5863,Fruit and Field,Cherry & Blackcurrant Wheat,Finkel & Garf Brewing Company,4.80,0,0.00,https://www.beeradvocate.com/beer/profile/3846...,['KT3418 from Colorado3/5\xa0\xa0rDev 0%look: ...


In [10]:
#checking if there are any reviewed beers without ratings
beer_df[beer_df['no_ratings'] == 0]

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review


In [11]:
#turn this column from str to float
beer_df['no_ratings'] = beer_df['no_ratings'].str.replace(',', '').astype(float)

In [12]:
empty_rating = beer_df[beer_df['no_ratings'] == 0]

In [13]:
type(beer_df['no_ratings'][5])

numpy.float64

In [14]:
beer_df.drop(empty_rating.index, inplace = True)

In [15]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,5915.0,3.42,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev..."
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029.0,3.19,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,3992.0,4.41,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%..."
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841.0,3.20,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,3799.0,3.91,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...
...,...,...,...,...,...,...,...,...
5649,Fruit and Field,Meggyes Favágó,Békésszentandrási Serfőzde (Agro Flott Bt.),5.00,1.0,3.74,https://www.beeradvocate.com/beer/profile/3293...,['joaopmgoncalves from Portugal3.74/5\xa0\xa0r...
5650,Fruit and Field,Skully,Rhinegeist Brewery,7.80,1.0,3.67,https://www.beeradvocate.com/beer/profile/3199...,['West_Chester_Ale_Tester from Ohio3.67/5\xa0\...
5651,Fruit and Field,Carney Juice,Independent Republic Brewing Co.,4.00,1.0,4.00,https://www.beeradvocate.com/beer/profile/5671...,['PaleAleWalt from New York4/5\xa0\xa0rDev 0%l...
5652,Fruit and Field,Cherry Ace,The Blue Elephant Craft Brew House,4.80,1.0,3.31,https://www.beeradvocate.com/beer/profile/3446...,['KrisDLSmith from Canada (ON)3.31/5\xa0\xa0rD...


In [ ]:
'@[^\s]+'  code for regex cleaning of data

### Cleaning the Reviews w/ Stopwords

In [111]:
stop_words = stopwords.words('english')
stop_words.extend(['look', 'smell', 'taste', 'feel', 'overall', 'rdev', 'beer', 'beers','nice','adams', 'jan', 'feb', 'mar',
                  'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec', 'canada', 'north'])
stop_words += string.punctuation



In [112]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
lemmatizer = WordNetLemmatizer()

In [113]:
# A function that will apply stop words, remove numbers, punctuation, and normalization to a body of text
def pre_process_text(text):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    def clean(doc):
        if doc is not None:
            stop_free = " ".join([i for i in doc.lower().split() if i not in stop and i.isalpha()])
            punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
            normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
            return normalized
        else:
            pass
    processed_text = clean(text)
    if processed_text is not None:
        return processed_text.split()

In [114]:
#takes in a dataframe with a column with raw text and outputs a new column with pre-processed text
def add_processed_text(df, output_column_name, column_to_target='review'):
    df[output_column_name] = df.apply(lambda row: pre_process_text(row[column_to_target]), axis=1)

In [115]:
def count_word_frequency(vectorizer, text):
    try:
        termFrequencyMatrix = vectorizer.fit_transform(text)
        return termFrequencyMatrix
    except:
        print('Problem in count word freq')
        print(traceback.format_exc())

In [116]:
#runs a vectorizer over a row of the target column and if empty it will pass. Afterwards it will 
#generate a new column with the term frequency matrix
def get_term_freq_matrices(vectorizer, df, target_column):
    df['tf_matrix'] = df.apply(lambda row: count_word_frequency(vectorizer, row[target_column]) if
    row[target_column] != None else None, axis=1)

In [117]:
#runs the processor over the review column and creates a new one with the results
add_processed_text(beer_df, 'clean_reviews', column_to_target='review')

In [ ]:
tfidf = TfidfVectorizer()

In [173]:
#runs the vectorizer over the df and column we just created with the processed text
get_term_freq_matrices(tfidf, beer_df, 'clean_reviews')

In [119]:
#creates a column that breaks the text out of a list and into a str
beer_df['naked_reviews'] = beer_df.apply(lambda row: ' '.join(row['clean_reviews']), axis =1)

In [141]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review,clean_reviews,tf_matrix,naked_reviews,tf_array
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,5915.0,3.42,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev...","[pours, little, cloudy, little, small, smell, ...","(0, 276)\t1.0\n (1, 213)\t1.0\n (2, 63)\t1...",pours little cloudy little small smell like fr...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029.0,3.19,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...,"[bottle, shaker, pint, hazy, copper, thick, wh...","(0, 44)\t1.0\n (1, 324)\t1.0\n (2, 279)\t1...",bottle shaker pint hazy copper thick white che...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,3992.0,4.41,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%...","[beautiful, ruby, red, head, dissipates, smell...","(0, 37)\t1.0\n (1, 435)\t1.0\n (2, 416)\t1...",beautiful ruby red head dissipates smell inten...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841.0,3.20,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...,"[one, favorite, summer, beer, smell, taste, bi...","(0, 347)\t1.0\n (1, 176)\t1.0\n (2, 484)\t...",one favorite summer beer smell taste bit like ...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,3799.0,3.91,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...,"[pour, pink, raspberry, impressive, white, hea...","(0, 299)\t1.0\n (1, 293)\t1.0\n (2, 312)\t...",pour pink raspberry impressive white head ligh...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5639,Fruit and Field,Mos'ka Garnatxa Beer,Cerveseria Birrart,6.00,1.0,3.67,https://www.beeradvocate.com/beer/profile/1953...,['GONZALOYANNA from Spain3.67/5\xa0\xa0rDev 0%...,"[timmermans, flute, enjoyed, reddish, amber, f...","(0, 50)\t1.0\n (1, 13)\t1.0\n (2, 8)\t1.0\...",timmermans flute enjoyed reddish amber foggy f...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5648,Fruit and Field,Rainier Raspberry Wheat,Harmon Restaurant & Brewery,?,1.0,2.60,https://www.beeradvocate.com/beer/profile/1386...,['barleywinefiend from Washington2.6/5\xa0\xa0...,"[poured, cloudy, straw, color, almost, head, w...","(0, 8)\t1.0\n (1, 1)\t1.0\n (2, 11)\t1.0\n...",poured cloudy straw color almost head white ma...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
5651,Fruit and Field,Carney Juice,Independent Republic Brewing Co.,4.00,1.0,4.00,https://www.beeradvocate.com/beer/profile/5671...,['PaleAleWalt from New York4/5\xa0\xa0rDev 0%l...,"[new, shandy, hint]","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 0)\t1.0",new shandy hint,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
5652,Fruit and Field,Cherry Ace,The Blue Elephant Craft Brew House,4.80,1.0,3.31,https://www.beeradvocate.com/beer/profile/3446...,['KrisDLSmith from Canada (ON)3.31/5\xa0\xa0rD...,[canada],"(0, 0)\t1.0",canada,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."


In [88]:
#turned empty lists into nan values after stopwords
beer_df.loc[~beer_df.clean_reviews.astype(bool),'clean_reviews']=np.nan

In [92]:
#checked the rows that were now empty
beer_df.isna().sum()

beer_style          0
beer_name           0
brewery_name        0
abv                 0
no_ratings          0
avg_rating          0
url                 0
review              0
clean_reviews    1466
tf_matrix        1466
naked_reviews       0
tf_array            0
dtype: int64

In [93]:
#dropped the rows with nan values
beer_df.dropna(subset=['clean_reviews'], inplace = True)

In [146]:
beer_df['naked_reviews'][0]

'pours little cloudy little small smell like fruity ale ipa taste pretty good hoppy pale perhaps say quite pale ale like average run mill bit fruity light hop flavor towards carbonation smooth like nitro relatively overall could drink em day sick would first choice looking drink beer two said see custom six pack fridge would okay adding one north definite rich color claim quite feel like need bit hoppy department even fruit hot day need something hoppy work quite well evening drink day dundee fifco usa portfolio beer one crossover recommendation someone looking get crafted beer granted bit pedestrian used today still delightful recently poured pint coloring may pique hue remind typical fruit pale ale listed get lot malt formerly recalled back front note sweet fruit ale get bit sticky sweetness factor entirely bottle year old poured squat pint fridge somewhat darker amber quite topped finger tam head recedes caramel malt lends sweet bready note toffee caramel plenty resinous hop bring h

In [373]:
beer_df

,beer_style,beer_name,brewery_name,abv,no_ratings,avg_rating,url,review,clean_reviews,tf_matrix,naked_reviews,tf_array
0,Fruit and Field,#9,Magic Hat Brewing Company,5.10,5915.0,3.42,https://www.beeradvocate.com/beer/profile/96/299/,"[""Elemental19xx from Illinois3.6/5\xa0\xa0rDev...","[pours, little, cloudy, little, small, smell, ...","(0, 276)\t1.0\n (1, 213)\t1.0\n (2, 63)\t1...",pours little cloudy little small smell like fr...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,Fruit and Field,Samuel Adams Cherry Wheat,Boston Beer Company (Samuel Adams),5.30,4029.0,3.19,https://www.beeradvocate.com/beer/profile/35/108/,['BergBeer from Virginia1.15/5\xa0\xa0rDev -63...,"[bottle, shaker, pint, hazy, copper, thick, wh...","(0, 44)\t1.0\n (1, 324)\t1.0\n (2, 279)\t1...",bottle shaker pint hazy copper thick white che...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Fruit and Field,Raspberry Tart,New Glarus Brewing Company,4.00,3992.0,4.41,https://www.beeradvocate.com/beer/profile/590/...,"[""TedHead from Illinois4.85/5\xa0\xa0rDev +10%...","[beautiful, ruby, red, head, dissipates, smell...","(0, 37)\t1.0\n (1, 435)\t1.0\n (2, 416)\t1...",beautiful ruby red head dissipates smell inten...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,Fruit and Field,Summer Shandy,Jacob Leinenkugel Brewing Company,4.20,3841.0,3.20,https://www.beeradvocate.com/beer/profile/710/...,['AlCaponeJunior from Texas2.79/5\xa0\xa0rDev ...,"[one, favorite, summer, beer, smell, taste, bi...","(0, 347)\t1.0\n (1, 176)\t1.0\n (2, 484)\t...",one favorite summer beer smell taste bit like ...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Fruit and Field,Rübæus,Founders Brewing Company,5.70,3799.0,3.91,https://www.beeradvocate.com/beer/profile/1199...,['TomQBigBeerGuy from Michigan3.88/5\xa0\xa0rD...,"[pour, pink, raspberry, impressive, white, hea...","(0, 299)\t1.0\n (1, 293)\t1.0\n (2, 312)\t...",pour pink raspberry impressive white head ligh...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5639,Fruit and Field,Mos'ka Garnatxa Beer,Cerveseria Birrart,6.00,1.0,3.67,https://www.beeradvocate.com/beer/profile/1953...,['GONZALOYANNA from Spain3.67/5\xa0\xa0rDev 0%...,"[timmermans, flute, enjoyed, reddish, amber, f...","(0, 50)\t1.0\n (1, 13)\t1.0\n (2, 8)\t1.0\...",timmermans flute enjoyed reddish amber foggy f...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5648,Fruit and Field,Rainier Raspberry Wheat,Harmon Restaurant & Brewery,?,1.0,2.60,https://www.beeradvocate.com/beer/profile/1386...,['barleywinefiend from Washington2.6/5\xa0\xa0...,"[poured, cloudy, straw, color, almost, head, w...","(0, 8)\t1.0\n (1, 1)\t1.0\n (2, 11)\t1.0\n...",poured cloudy straw color almost head white ma...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
5651,Fruit and Field,Carney Juice,Independent Republic Brewing Co.,4.00,1.0,4.00,https://www.beeradvocate.com/beer/profile/5671...,['PaleAleWalt from New York4/5\xa0\xa0rDev 0%l...,"[new, shandy, hint]","(0, 1)\t1.0\n (1, 2)\t1.0\n (2, 0)\t1.0",new shandy hint,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
5652,Fruit and Field,Cherry Ace,The Blue Elephant Craft Brew House,4.80,1.0,3.31,https://www.beeradvocate.com/beer/profile/3446...,['KrisDLSmith from Canada (ON)3.31/5\xa0\xa0rD...,[canada],"(0, 0)\t1.0",canada,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."


### test run

In [348]:
df = beer_df[['abv', 'avg_rating', 'naked_reviews']]

In [349]:
df['abv'] = df['abv'].replace(['?'], [0])

<ipython-input-349-83af63e261ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abv'] = df['abv'].replace(['?'], [0])


In [350]:
zeros = df[df['abv'] == 0]

In [351]:
df.drop(zeros.index, inplace = True)

C:\Users\GMoneyMan\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [357]:
df

,abv,avg_rating,naked_reviews
0,5.10,3.0,pours little cloudy little small smell like fr...
1,5.30,3.0,bottle shaker pint hazy copper thick white che...
2,4.00,4.0,beautiful ruby red head dissipates smell inten...
3,4.20,3.0,one favorite summer beer smell taste bit like ...
4,5.70,4.0,pour pink raspberry impressive white head ligh...
...,...,...,...
5636,5.00,4.0,bottle purchased la crécelle brewery tasted sl...
5638,4.50,4.0,bright color fizzy white ton blood orange tast...
5639,6.00,4.0,timmermans flute enjoyed reddish amber foggy f...
5651,4.00,4.0,new shandy hint


In [353]:
df['avg_rating'] = df['avg_rating'].round()

<ipython-input-353-9211e852e62a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['avg_rating'] = df['avg_rating'].round()


In [354]:
df['avg_rating'].value_counts()

4.0    2388
3.0    1166
2.0     171
5.0      60
1.0      11
Name: avg_rating, dtype: int64

In [355]:
x = df['naked_reviews']
y = df['avg_rating']

In [356]:
#running train / test / split on naked reviews (str)
data_train, data_test, target_train, target_test = train_test_split(x, y, test_size=0.20, random_state=21)

In [358]:
tfidf = TfidfVectorizer(max_df = .9, min_df=.1)

In [359]:
tfidf_data_train = tfidf.fit_transform(data_train)
tfidf_data_test = tfidf.transform(data_test)

print(tfidf_data_train.shape)
print(tfidf_data_test.shape)

(3036, 236)
(760, 236)


In [361]:
data_train

3204    pleased berry component prominent giving tart ...
969     canada hoppypocket bottled jun cloudy dark cop...
939     new oz expires dec sampled september pours str...
2065    south beer answer poured snifter pink color cl...
1048    definitely first beer long bomber bottle reall...
                              ...                        
48      pint brown ruby thin head becomes leaving litt...
811     confused razberry two different poured glowing...
2130    tap bruery terreux tasting room fruit beer coc...
1340    oz bottle pours dark tinged lightly murky ambe...
4116    oz sampler light pink color look almost like p...
Name: naked_reviews, Length: 3036, dtype: object

In [360]:
non_zero_cols = tfidf_data_train.nnz / float(tfidf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tfidf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 44.85935441370224
Percentage of columns containing 0: 0.8099179897724481


In [302]:
tfidf_data_train

<3036x236 sparse matrix of type '<class 'numpy.float64'>'
	with 136193 stored elements in Compressed Sparse Row format>

In [303]:
target_train

3204    4.0
969     3.0
939     3.0
2065    4.0
1048    3.0
       ... 
48      4.0
811     4.0
2130    4.0
1340    3.0
4116    3.0
Name: avg_rating, Length: 3036, dtype: float64

In [318]:
rf = RandomForestClassifier(random_state = 21)#max_depth=10,max_features=10)
rf.fit(tfidf_data_train, target_train)

RandomForestClassifier(random_state=21)

In [319]:
rf_test_preds = rf.predict(tfidf_data_test)

In [320]:
rf_recall = recall_score(target_test, rf_test_preds, average='weighted')
rf_acc_score = accuracy_score(target_test, rf_test_preds)
rf_f1_score = f1_score(target_test, rf_test_preds, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(rf_recall))
print("Testing Accuracy: {:.4}".format(rf_acc_score))
print("F1 Score: {:.4}".format(rf_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.6816
Testing Accuracy: 0.6816
F1 Score: 0.6285


In [311]:
''' XGBoost '''
from xgboost import XGBClassifier

In [362]:
xgb = XGBClassifier()
xgb.fit(tfidf_data_train, target_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [365]:
xgb_test_preds = xgb.predict(tfidf_data_test)

In [366]:
xgb_recall = recall_score(target_test, xgb_test_preds, average='weighted')
xgb_acc_score = accuracy_score(target_test, xgb_test_preds)
xgb_f1_score = f1_score(target_test, xgb_test_preds, average='weighted')

print('XGBOOST with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(xgb_recall))

print("Testing Accuracy: {:.4}".format(xgb_acc_score))
print("F1 Score: {:.4}".format(xgb_f1_score))

XGBOOST with Stematized Features:
---------------
Recall: 0.6671
Testing Accuracy: 0.6671
F1 Score: 0.6305


In [368]:
#scale X_train and X_test for svm

scaler = StandardScaler()
scaled_data_train = scaler.fit_transform(tfidf_data_train)
scaled_data_test = scaler.transform(tfidf_data_test)

ValueError: could not convert string to float: 'pleased berry component prominent giving tart emphasis blonde refreshing nicely'

In [369]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')
model_log = logreg.fit(tfidf_data_train, target_train)

In [370]:
log_test_pred = model_log.predict(tfidf_data_test)

In [372]:
log_recall = recall_score(target_test, log_test_pred, average='weighted')
log_acc_score = accuracy_score(target_test, log_test_pred)
log_f1_score = f1_score(target_test, log_test_pred, average='weighted')
print('Random Forest with Stematized Features:')
print(3*'-----')

print('Recall: {:.4}'.format(log_recall))

print("Testing Accuracy: {:.4}".format(log_acc_score))
print("F1 Score: {:.4}".format(log_f1_score))

Random Forest with Stematized Features:
---------------
Recall: 0.5882
Testing Accuracy: 0.5882
F1 Score: 0.5991


### end of test

In [41]:
beer_df['tf_array'] = beer_df.apply(lambda row: row['tf_matrix'].toarray(), axis = 1)

In [59]:
type(beer_df['tf_matrix'][0])

scipy.sparse.csr.csr_matrix

In [60]:
type(beer_df['tf_array'][0])

numpy.ndarray

In [ ]:
#running the clean_reviews column through tfidf
beer_l = beer_df['clean_reviews'].values.tolist()
type(beer_l)

In [ ]:
collected_c_r = count_word_frequency(tfidf, beer_l)

In [ ]:
#running the naked_reviews column through tfidf
beer_r = beer_df['naked_reviews'].values.tolist()
type(beer_r)

In [ ]:
collected_n_r = count_word_frequency(tfidf, beer_r)

In [ ]:
type(collected_n_r)

In [49]:
type(beer_df['avg_rating'][0])

numpy.float64

In [34]:
type(beer_df['clean_reviews'][0])

list

In [35]:
type(beer_df['naked_reviews'][0])

str

In [54]:
text_tf = tfidf.fit_transform(beer_df['tf_matrix'])

AttributeError: lower not found

In [202]:
beer_df['naked_reviews']

0       pours little cloudy little small smell like fr...
1       bottle shaker pint hazy copper thick white che...
2       beautiful ruby red head dissipates smell inten...
3       one favorite summer beer smell taste bit like ...
4       pour pink raspberry impressive white head ligh...
                              ...                        
5639    timmermans flute enjoyed reddish amber foggy f...
5648    poured cloudy straw color almost head white ma...
5651                                      new shandy hint
5652                                               canada
5653                                                north
Name: naked_reviews, Length: 4077, dtype: object

In [238]:
#running train / test / split on naked reviews (str)
data_train, data_test, target_train, target_test = train_test_split(beer_df['naked_reviews'], beer_df['avg_rating'].values, test_size=0.20, random_state=21)

In [236]:
#running train / test/ split on clean_reviews (list)
data_train, data_test, target_train, target_test = train_test_split(beer_df['clean_reviews'], beer_df['avg_rating'].values, test_size=0.20, random_state=21)

In [191]:
tfidf = TfidfVectorizer(max_df = .9, min_df=.1)

In [239]:
tfidf_data_train = tfidf.fit_transform(data_train)
tfidf_data_test = tfidf.transform(data_test)

print(tfidf_data_train.shape)
print(tfidf_data_test.shape)

(3261, 226)
(816, 226)


In [203]:
non_zero_cols = tfidf_data_train.nnz / float(tfidf_data_train.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))

percent_sparse = 1 - (non_zero_cols / float(tfidf_data_train.shape[1]))
print('Percentage of columns containing 0: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 42.61729530818767
Percentage of columns containing 0: 0.8114278968664262


In [223]:
tfidf_data_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.12002252, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.15972417, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [233]:
tfidf_data_train.reshape(-1, 1).data

array([0.69485207, 0.7191527 , 0.34586865, ..., 0.14072918, 0.07282541,
       1.        ])

In [235]:
print(type(tfidf_data_train.toarray()))
tfidf_data_train.data

<class 'numpy.ndarray'>


array([0.69485207, 0.7191527 , 0.34586865, ..., 0.14072918, 0.07282541,
       1.        ])

In [225]:
target_train

array([3.5 , 2.99, 3.74, ..., 3.67, 3.46, 3.51])

In [230]:
rf = RandomForestClassifier(random_state = 21)#max_depth=10,max_features=10)
rf.fit(tfidf_data_train.data,target_train)

ValueError: Expected 2D array, got 1D array instead:
array=[0.69485205 0.7191527  0.34586865 ... 0.14072917 0.0728254  1.        ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [182]:
type(data_train)

pandas.core.series.Series

array([0.0681992 , 0.0416199 , 0.32642651, ..., 0.1714665 , 0.16462331,
       0.10582308])

In [187]:
rf = RandomForestClassifier(random_state = 21)#max_depth=10,max_features=10)
rf.fit(tfidf_data_train,target_train)

ValueError: Unknown label type: 'continuous'

In [46]:
rf = RandomForestClassifier(random_state = 21)#max_depth=10,max_features=10)
rf.fit(data_train,target_train)

ValueError: setting an array element with a sequence.

1) tts on df
2) take either lem or stem for both train and test and put in new var
3) tfidf fit transform on train and transform on test
4) put sparse tfidf to array (to dense maybe) (1 for test and 1 for train) (to allow concat)
it will be number of obs by the number of unique words (columns will be the same for both since we used fit transform for train as test) 
5) concat train tfidf to train columns not including text or the lem or stem, and same for test
6) run RF

In [67]:
'''1: tts on df, using reviews that have already been lemmitized'''

data_train, data_test, target_train, target_test = train_test_split(beer_df['naked_reviews'] ,beer_df['avg_rating'].values, test_size=0.20, random_state=21)

In [160]:
'''2: lemitize for both train and test and put into a new var
   3: tfidf fit transform on train and transform on test'''

#Here we will use the pre_process_text function to put the training and testing data through

tfidf_data_train = tfidf.fit_transform(data_train)
tfidf_data_test = tfidf.transform(data_test)

tfidf_data_test.data



array([0.0681992 , 0.0416199 , 0.32642651, ..., 0.1714665 , 0.16462331,
       0.10582308])

In [189]:
tfidf_data_test.shape

(816, 226)

In [184]:
'''4: put sparse tfidf to array (to dense maybe) (1 for test and 1 for train) (to allow concat)
it will be number of obs by the number of unique words (columns will be the same for both since'''

tfidf_data_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
'''5: concat train tfidf to train columns not including text or the lem or stem, and same for test'''




In [165]:
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(data_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(target_test, y_pred))

In [166]:
model_Evaluate(rf)

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

### scrap

In [ ]:
#funtion to lower text
def to_lower(text):
    return ' '.join([w.lower() for w in word_tokenize(text)])

def remove_number(text):
    return ' '.join(c for c in text if not c.isdigit())

def process_article(article):
    tokens = nltk.word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in sw_set]
    return stopwords_removed 
    

In [ ]:
beer_df['cleaned_review'] = beer_df['review'].apply(pre_process_text)

In [ ]:
beer_df

In [ ]:
beer_df['naked_reviews'] = beer_df.apply(lambda row: ' '.join(row['cleaned_review']), axis =1)

In [ ]:
beer_df

In [79]:
review1 = beer_df['review'][0]

In [ ]:
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+') #tokenize just normal characters and numbers

tokenized_review = tokenizer.tokenize(review1)

In [ ]:
def filtering(words):
    filtered_review = []
    for x in words:
        if x not in stop_words:
            filtered_review.append(x)
            print(filtered_review)
    return filtered_review

        

In [ ]:
f_review = filtering(tokenized_review)
f_review

In [ ]:
fdist = FreqDist(filtering(tokenized_review))
plt.figure(figsize=(10,10))
fdist.plot(10)

### Stemming

In [ ]:
ps = PorterStemmer()

stemmed_review=[]
for w in f_review:
    stemmed_review.append(ps.stem(w))

print(stemmed_review)

### Lemmatize

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
review1

In [ ]:
def pre_process_documents(abstracts):
    output = {}
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()

    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized
    for pmid, abstract in abstracts.items():
        output[pmid] = clean(abstract).split()
    return output

In [ ]:
pre_process_documents({n:k for n, k in enumerate(f_review)})

In [ ]:
def pre_process_text(text):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    def clean(doc):
        if doc is not None:
            stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
            punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
            normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
            return normalized
        else:
            pass
    processed_text = clean(text)
    if processed_text is not None:
        return processed_text.split()

In [126]:
pre_process_text(review1)

['pours',
 'little',
 'cloudy',
 'little',
 'small',
 'smell',
 'like',
 'fruity',
 'ale',
 'ipa',
 'taste',
 'pretty',
 'good',
 'hoppy',
 'pale',
 'perhaps',
 'say',
 'quite',
 'pale',
 'ale',
 'like',
 'average',
 'run',
 'mill',
 'bit',
 'fruity',
 'light',
 'hop',
 'flavor',
 'towards',
 'carbonation',
 'smooth',
 'like',
 'nitro',
 'relatively',
 'overall',
 'could',
 'drink',
 'em',
 'day',
 'sick',
 'would',
 'first',
 'choice',
 'looking',
 'drink',
 'beer',
 'two',
 'said',
 'see',
 'custom',
 'six',
 'pack',
 'fridge',
 'would',
 'okay',
 'adding',
 'one',
 'north',
 'definite',
 'rich',
 'color',
 'claim',
 'quite',
 'feel',
 'like',
 'need',
 'bit',
 'hoppy',
 'department',
 'even',
 'fruit',
 'hot',
 'day',
 'need',
 'something',
 'hoppy',
 'work',
 'quite',
 'well',
 'evening',
 'drink',
 'day',
 'dundee',
 'fifco',
 'usa',
 'portfolio',
 'beer',
 'one',
 'crossover',
 'recommendation',
 'someone',
 'looking',
 'get',
 'crafted',
 'beer',
 'granted',
 'bit',
 'pedestrian

### TF-IDF

In [23]:
tfidf = TfidfVectorizer()

In [ ]:
X_lem = beer_df['tf_matrix']

In [ ]:
y_lem = beer_df['avg_rating']

In [ ]:
X_train_lem, X_test_lem, y_train_lem, y_test_lem = train_test_split(X_lem, y_lem, test_size=0.20, random_state=1)
tfidf = TfidfVectorizer()
#tfidf = TfidfTransformer()

tfidf_data_train_lem = tfidf.fit_transform(X_train_lem)
tfidf_data_test_lem = tfidf.transform(X_test_lem)

tfidf_data_train_lem

### Processing

In [ ]:
#takes in a dataframe with a column with raw text and outputs a new column with pre-processed text
def add_processed_text(df, output_column_name, column_to_target='review'):
    df[output_column_name] = df.apply(lambda row: pre_process_text(row[column_to_target]), axis=1)

In [ ]:
def count_word_frequency(vectorizer, text):
    try:
        termFrequencyMatrix = vectorizer.fit_transform(text)
        return termFrequencyMatrix
    except:
        print('Problem in count word freq')
        print(traceback.format_exc())
        #print(text)

In [ ]:
#runs a vectorizer over a row of the target column and if empty it will pass. Afterwards it will 
#generate a new column with the term frequency matrix
def get_term_freq_matrices(vectorizer, df, target_column):
    df['tf_matrix'] = df.apply(lambda row: count_word_frequency(vectorizer, row[target_column]) if
    row[target_column] != None else None, axis=1)

In [ ]:
add_processed_text(beer_df, 'clean_reviews', column_to_target='review')

In [ ]:
get_term_freq_matrices(tfidf, beer_df, 'clean_reviews')

In [ ]:
beer_df

In [ ]:
beer_df['tf_matrix'] = beer_df.apply(lambda row: row['tf_matrix'].toarray(), axis = 1)

In [ ]:
beer_df['tf_matrix']

### Applying Models to the Matrix

In [36]:
data_train, data_test, target_train, target_test = train_test_split(beer_df['naked_reviews'] ,beer_df['avg_rating'].values, test_size=0.20, random_state=21)

In [37]:
target_train

array([3.56, 3.49, 3.94, ..., 3.9 , 2.74, 3.5 ])

In [38]:
data_train

3252    bmaggot lithuania3565xa0xa0rdev 0look 3 smell ...
4273    jtuomi finland3495xa0xa0rdev 0look 4 smell 375...
4026    nerofiddled pennsylvania3945xa0xa0rdev 0look 3...
4562    beaver13 colorado3585xa0xa0rdev 0look 4 smell ...
2985    georgeblazer pennsylvania3715xa0xa0rdev 77look...
                              ...                        
4801    williethreebiers connecticut3825xa0xa0rdev 0lo...
1166    phyl21ca canada qc2275xa0xa0rdev 275look 3 sme...
48      jwc215 arizona3935xa0xa0rdev 08look 375 smell ...
787     captaincoffee virginia2985xa0xa0rdev 88look 35...
5434    lennyovies mexico355xa0xa0rdev 0look 35 smell ...
Name: naked_reviews, Length: 4434, dtype: object

In [39]:
rf = RandomForestClassifier(random_state = 21)#max_depth=10,max_features=10)
rf.fit(data_train,target_train)

ValueError: could not convert string to float: 'bmaggot lithuania3565xa0xa0rdev 0look 3 smell 375 taste 35 feel 375 overall 35dec 11 2019'

### Collecting the tf-idf of all beers

In [ ]:
type(beer_df['naked_reviews'][0:2])

In [ ]:
beer_r = beer_df['naked_reviews'].values.tolist()
type(beer_r)

In [ ]:
type(beer_r[0])

In [ ]:
collected_words = count_word_frequency(tfidf, beer_r)

In [ ]:
type(collected_words)

In [ ]:
beer_df['naked_reviews'].values.tolist()